# Практические задания к лекции № 5

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [4]:
from sklearn.model_selection import GridSearchCV

In [5]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [6]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [8]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [9]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [10]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [11]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [12]:
pipeline_1 = Pipeline([
    ('features',feats),
    ('RF_classifier', RandomForestClassifier(random_state = 28)),
])

pipeline_2 = Pipeline([
    ('features',feats),
    ('LR_classifier', LogisticRegression(random_state = 28)),
])

pipeline_3 = Pipeline([
    ('features',feats),
    ('XGB_classifier', XGBClassifier(verbosity = 0, random_state = 28, use_label_encoder=False)),
])

In [13]:
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [14]:
pipeline_2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [15]:
pipeline_3.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [16]:
preds_1 = pipeline_1.predict_proba(X_test)[:, 1]
preds_1[:10]

array([0.34, 0.22, 0.21, 0.02, 0.  , 0.76, 0.03, 0.14, 0.14, 0.73])

In [17]:
preds_2 = pipeline_2.predict_proba(X_test)[:, 1]
preds_2[:10]

array([0.22805865, 0.3350657 , 0.15347886, 0.12466446, 0.15507743,
       0.6431308 , 0.06214346, 0.077948  , 0.36717132, 0.76751542])

In [18]:
preds_3 = pipeline_3.predict_proba(X_test)[:, 1]
preds_3[:10]

array([0.47611395, 0.23087966, 0.0737261 , 0.03141731, 0.02561874,
       0.9679959 , 0.06951059, 0.2885457 , 0.11367497, 0.4902508 ],
      dtype=float32)

Сравним результаты моделей на основе fscore при разных $\beta$. Так как нам важно максимизировать прибыль, то полнота более важна, чем точность, а значит $\beta$ возьмем больше 1. Добавим TP, FP,	FN для решения вопроса 3.

In [19]:
models_results = {
    'classifier': [],
    'beta': [],
    'thresholds': [],
    'fscore': [],
    'precision': [],
    'recall': [],
    'TP': [],
    'FP': [],
    'FN': []
}

In [20]:
def mistakes_matrix(y_true, y_pred):
    
    TP = sum(y_pred - 2*y_true == -1)
    FP = sum(y_pred - y_true == 1)
    FN = sum(y_pred - y_true == -1)
    TN = sum(y_pred - 2*y_true == 0)
    
    return(TP, FP, FN, TN)

In [21]:
for preds, clf in zip([preds_1, preds_2, preds_3], ['RandomForestClassifier', 'LogisticRegression', 'XGBClassifier']):
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    #mod_b = []
    for b in [1, 1.5, 2, 2.5, 3, 5]:

        fscore = (1 + b**2) * precision * recall / ((b**2) * precision + recall)

        # locate the index of the largest f score
        ix = np.argmax(fscore)
        models_results['classifier'].append(clf)
        models_results['beta'].append(b)
        models_results['thresholds'].append(thresholds[ix])
        models_results['fscore'].append(fscore[ix])
        models_results['precision'].append(precision[ix])
        models_results['recall'].append(recall[ix])
        
        mist_mat = mistakes_matrix(y_test, preds>thresholds[ix])
        models_results['TP'].append(mist_mat[0])
        models_results['FP'].append(mist_mat[1])
        models_results['FN'].append(mist_mat[2])

models_comp = pd.DataFrame(data=models_results).sort_values('fscore', ascending=False)
models_comp

,classifier,beta,thresholds,fscore,precision,recall,TP,FP,FN
17,XGBClassifier,5.0,0.020589,0.885212,0.261830,0.978389,497,1404,12
5,RandomForestClassifier,5.0,0.020000,0.879235,0.226095,0.994106,502,1619,7
11,LogisticRegression,5.0,0.009638,0.869228,0.203600,1.000000,508,1991,1
16,XGBClassifier,3.0,0.050889,0.780472,0.326162,0.923379,469,971,40
4,RandomForestClassifier,3.0,0.130000,0.771076,0.358566,0.884086,443,756,66
15,XGBClassifier,2.5,0.087351,0.746674,0.385677,0.878193,446,712,63
10,LogisticRegression,3.0,0.089543,0.741202,0.258555,0.935167,475,1365,34
3,RandomForestClassifier,2.5,0.130000,0.735418,0.358566,0.884086,443,756,66
14,XGBClassifier,2.0,0.106143,0.704590,0.412098,0.856582,435,622,74
9,LogisticRegression,2.5,0.133875,0.695575,0.305422,0.874263,444,1012,65


<b>Вопрос 2: предположим, что на удержание одного пользователя у нас уйдет 1 доллар. При этом средняя ожидаемая прибыль с каждого TP (true positive) - 2 доллара. Оцените качество модели выше с учетом этих данных и ответьте на вопрос, является ли она потенциально экономически целесообразной?</b>

Доход = TP   
Затраты = TP + FN  
Прибыль = Доход - Затраты     
Упущенная прибыль = FP

In [22]:
models_comp['Income'] = 2 * models_comp.TP
models_comp['Expenses'] = models_comp.TP +  models_comp.FN
models_comp['Profit'] = models_comp.Income - models_comp.Expenses
models_comp['LostProfits'] = 2 * models_comp.FP

In [23]:
models_comp.sort_values('Profit', ascending=False)

,classifier,beta,thresholds,fscore,precision,recall,TP,FP,FN,Income,Expenses,Profit,LostProfits
11,LogisticRegression,5.0,0.009638,0.869228,0.203600,1.000000,508,1991,1,1016,509,507,3982
5,RandomForestClassifier,5.0,0.020000,0.879235,0.226095,0.994106,502,1619,7,1004,509,495,3238
17,XGBClassifier,5.0,0.020589,0.885212,0.261830,0.978389,497,1404,12,994,509,485,2808
10,LogisticRegression,3.0,0.089543,0.741202,0.258555,0.935167,475,1365,34,950,509,441,2730
16,XGBClassifier,3.0,0.050889,0.780472,0.326162,0.923379,469,971,40,938,509,429,1942
15,XGBClassifier,2.5,0.087351,0.746674,0.385677,0.878193,446,712,63,892,509,383,1424
8,LogisticRegression,2.0,0.133875,0.636988,0.305422,0.874263,444,1012,65,888,509,379,2024
9,LogisticRegression,2.5,0.133875,0.695575,0.305422,0.874263,444,1012,65,888,509,379,2024
3,RandomForestClassifier,2.5,0.130000,0.735418,0.358566,0.884086,443,756,66,886,509,377,1512
4,RandomForestClassifier,3.0,0.130000,0.771076,0.358566,0.884086,443,756,66,886,509,377,1512


Все модели оказались экономически целесообразны, лучше всего себя показала модель LogisticRegression при выборе порога для f score при $ \beta = 5 $. Однако упущенная прибыль значительна, значит у этой модели есть большой потенциал.

**Подбор гиперпараметров**

Посмотрим на важность признаков, для этого используем l1 регуляризацию

In [24]:
pipeline_4 = Pipeline([
    ('features',feats),
    ('LR_classifier', LogisticRegression(penalty='l1', solver='liblinear', random_state = 28))
    ])

In [25]:
from sklearn.metrics import fbeta_score, make_scorer

In [26]:
f5_scorer = make_scorer(fbeta_score, beta=5)

In [27]:
pipeline_4.fit(X_train, y_train)
coef = pipeline_4['LR_classifier'].coef_[0]
coef

array([-0.75924228,  0.07335236, -0.63296428,  0.        , -0.54875475,
        0.06205322,  0.13149705, -0.01091211,  0.04477772,  0.0490384 ,
        0.        ,  0.        , -0.23299438, -0.06484819, -0.16691417,
       -0.14601382,  0.        , -0.02058708,  0.        , -1.00800155,
       -0.08868956,  0.74030375,  0.16047902, -0.09001255,  0.03799066])

Видно, что коэффициенты у некоторых признаков зануляются, поэтому добавим в pipeline отбор признаков

In [28]:
param = {'penalty' : ['l1', 'l2'],
        'C' : np.logspace(-4, 4, 20),
        'class_weight': [None, 'balanced'],
        'solver': ['liblinear']}

lr_model = LogisticRegression(random_state = 28)

In [29]:
from sklearn.feature_selection import SelectFromModel

In [30]:
pipeline_5 = Pipeline([
    ('features',feats),
    ('feature_selection', SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear', random_state = 28))),
    ('gs_model', GridSearchCV(estimator=lr_model, param_grid=param, scoring='f1'))
    ])

In [31]:
pipeline_5.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [32]:
pd.DataFrame(pipeline_5['gs_model'].cv_results_).sort_values('rank_test_score', ascending=True)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
22,0.006216,0.000715,0.001070,0.000092,0.012743,balanced,l1,liblinear,"{'C': 0.012742749857031334, 'class_weight': 'b...",0.495726,0.504926,0.524510,0.488169,0.493252,0.501317,0.012807,1
26,0.007991,0.000632,0.001288,0.000370,0.033598,balanced,l1,liblinear,"{'C': 0.03359818286283781, 'class_weight': 'ba...",0.504202,0.501199,0.519481,0.483832,0.486094,0.498962,0.013023,2
19,0.005953,0.000550,0.001412,0.000477,0.004833,balanced,l2,liblinear,"{'C': 0.004832930238571752, 'class_weight': 'b...",0.507246,0.492754,0.517900,0.484108,0.487148,0.497831,0.012803,3
15,0.005604,0.000526,0.001040,0.000085,0.001833,balanced,l2,liblinear,"{'C': 0.0018329807108324356, 'class_weight': '...",0.507975,0.495747,0.509709,0.480601,0.488943,0.496595,0.011105,4
11,0.005656,0.000379,0.001125,0.000257,0.000695,balanced,l2,liblinear,"{'C': 0.0006951927961775605, 'class_weight': '...",0.507975,0.492718,0.504222,0.481108,0.488943,0.494994,0.009882,5


Лучшие параметры:

In [33]:
pipeline_5['gs_model'].best_params_

{'C': 0.012742749857031334,
 'class_weight': 'balanced',
 'penalty': 'l1',
 'solver': 'liblinear'}

Итоговый pipeline:

In [34]:
pipeline_6 = Pipeline([
    ('features',feats),
    ('feature_selection', SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear', random_state = 28))),
    ('LR_classifier', LogisticRegression(**pipeline_5['gs_model'].best_params_, random_state = 28))
    ])

In [35]:
pipeline_6.fit(X_train, y_train)
preds_6 = pipeline_6.predict_proba(X_test)[:, 1]
preds_6[:10]

array([0.49768564, 0.55920217, 0.43562089, 0.33819229, 0.47659579,
       0.81495863, 0.23617983, 0.26100328, 0.74539518, 0.87685655])

In [36]:
fin_models_results = {
    'classifier': [],
    'beta': [],
    'thresholds': [],
    'fscore': [],
    'precision': [],
    'recall': [],
    'TP': [],
    'FP': [],
    'FN': []
}

In [37]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_6)
precision= np.where(precision==0., 1e-10, precision)
recall= np.where(recall==0., 1e-10, recall)

b = 5
fscore = (1 + b**2) * precision * recall / ((b**2) * precision + recall)

# locate the index of the largest f score
ix = np.argmax(fscore)
fin_models_results['classifier'].append('LogisticRegression_tune')
fin_models_results['beta'].append(b)
fin_models_results['thresholds'].append(thresholds[ix])
fin_models_results['fscore'].append(fscore[ix])
fin_models_results['precision'].append(precision[ix])
fin_models_results['recall'].append(recall[ix])

mist_mat = mistakes_matrix(y_test, preds>thresholds[ix])
fin_models_results['TP'].append(mist_mat[0])
fin_models_results['FP'].append(mist_mat[1])
fin_models_results['FN'].append(mist_mat[2])

models_comp_2 = pd.DataFrame(data=fin_models_results).sort_values('fscore', ascending=False)
models_comp_2

,classifier,beta,thresholds,fscore,precision,recall,TP,FP,FN
0,LogisticRegression_tune,5,0.076463,0.869228,0.2036,1.0,451,777,58


In [38]:
models_comp_2['Income'] = 2 * models_comp.TP
models_comp_2['Expenses'] = models_comp.TP +  models_comp.FN
models_comp_2['Profit'] = models_comp.Income - models_comp.Expenses
models_comp_2['LostProfits'] = 2 * models_comp.FP

Результат:

In [39]:
models_comp_2

,classifier,beta,thresholds,fscore,precision,recall,TP,FP,FN,Income,Expenses,Profit,LostProfits
0,LogisticRegression_tune,5,0.076463,0.869228,0.2036,1.0,451,777,58,656,509,147,374


In [40]:
models_comp.sort_values('Profit', ascending=False)[:5]

,classifier,beta,thresholds,fscore,precision,recall,TP,FP,FN,Income,Expenses,Profit,LostProfits
11,LogisticRegression,5.0,0.009638,0.869228,0.203600,1.000000,508,1991,1,1016,509,507,3982
5,RandomForestClassifier,5.0,0.020000,0.879235,0.226095,0.994106,502,1619,7,1004,509,495,3238
17,XGBClassifier,5.0,0.020589,0.885212,0.261830,0.978389,497,1404,12,994,509,485,2808
10,LogisticRegression,3.0,0.089543,0.741202,0.258555,0.935167,475,1365,34,950,509,441,2730
16,XGBClassifier,3.0,0.050889,0.780472,0.326162,0.923379,469,971,40,938,509,429,1942


Модель получилась гораздо хуже той, что работает по умолчанию